# <center><u>RUN SWING SOLEIL octobre24</u></center>

In [8]:

from SwingTools import extract_from_h5 as extract
from SwingTools import integrate as integ
from SwingTools import save_exported_data
#from SwingTools import extract_sample_names
from SwingTools import plot_integration
from SwingTools import create_info_file
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
#import fabio
import glob
import os
import logging


In [9]:
# folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
# folder = 'D:/swing/kin_11_ag_55deg'
#folder = 'D:/swing/kin_21_au_500tes'
folder = 'D:/swing/test'
df, asso = create_info_file(folder)


Error on:  []


In [11]:
asso.head()

,File_sample,Sample name_sample,Mean Transmission_sample,Type_sample,Channel,Position,Time_sample,Flow_sample,File_ref,Sample name_ref,Mean Transmission_ref,Type_ref,Time_ref,Flow_ref
0,rodriguez_02263_2024-10-11_16-47-31.h5,S_C_1_P_1,0.605944,sample,1,1,2024-10-11 16:47:31,unknown,rodriguez_02199_2024-10-11_16-28-28.h5,R_C_1_P_1,0.774944,ref,2024-10-11 16:28:28,unknown
1,rodriguez_02268_2024-10-11_16-48-25.h5,S_C_1_P_6,0.591169,sample,1,6,2024-10-11 16:48:25,unknown,rodriguez_02204_2024-10-11_16-29-22.h5,R_C_1_P_6,0.765734,ref,2024-10-11 16:29:22,unknown
2,rodriguez_02527_2024-10-11_17-32-33.h5,S_C_2_SF_5_P_10,0.581667,sample,2,10,2024-10-11 17:32:33,SF,rodriguez_02230_2024-10-11_16-33-54.h5,R_C_2_P_10,0.768632,ref,2024-10-11 16:33:54,unknown


In [45]:

import os

maskfile = 'mask.edf'
coef = 0.0181 

for index, row in asso.iterrows():
    sample_file = str(row["File_sample"])
    ref_file = str(row["File_ref"])
    sample_path = folder+'/'+sample_file
    ref_path = folder+'/'+ref_file
    #extract data for sample and associated reference
    sample_name, sample_eiger, sample_eiger_mean, sample_basler_image, sample_position, sample_params, sample_time_stamps, sample_transmission = extract(sample_path,print_output=False)
    ref_name, ref_eiger, ref_eiger_mean, ref_basler_image, ref_position, ref_params, ref_time_stamps, ref_transmission = extract(ref_path,print_output=False)
    #Do integration
    sample_integration_mean = integ(sample_eiger_mean,sample_params,maskfile)
    ref_integration_mean = integ(ref_eiger_mean,ref_params,maskfile)

    sample_q_iso = sample_integration_mean[0]
    sample_i_iso = sample_integration_mean[1]

    ref_q_iso = ref_integration_mean[0]
    ref_i_iso = ref_integration_mean[1]

    #apply correction
    corr_i_iso = coef*((sample_i_iso/np.mean(sample_transmission))-(ref_i_iso/np.mean(ref_transmission)))

    # Create a new directory with the sample name
    # file_num = sample_file.split('rodriguez_')[1]
    # file_num = sample_file.split('_2024')[0]
    file_num = sample_file.split('rodriguez_')[1].split('_')[0]
    sample_dir = os.path.join(folder, sample_name+'__'+file_num)
    if not os.path.exists(sample_dir):
        os.makedirs(sample_dir)


        # ###########
        # file_num = sample_file.split('rodriguez_')[1]
        # file_num = sample_file.split('_2024')[0]
        # output_dir = output_dir + '_'+ file_num
        # os.makedirs(output_dir, exist_ok=True)
        # ###########

    #save to files
    sample_integration_col = np.column_stack(( sample_q_iso,  sample_i_iso))
    ref_integration_col = np.column_stack((ref_q_iso, ref_i_iso))
    corr_integration_col = np.column_stack((sample_q_iso, corr_i_iso))

    np.save(os.path.join(sample_dir, f'sample_integration_{sample_name}.npy'), sample_integration_col)
    np.save(os.path.join(sample_dir, f'ref_integration_{sample_name}.npy'), ref_integration_col)
    np.save(os.path.join(sample_dir, f'corr_integration_{sample_name}.npy'), corr_integration_col)

    header = "sample_q_iso   sample_i_iso"
    np.savetxt(os.path.join(sample_dir, f'sample_integration_{sample_name}.txt'), sample_integration_col,header = header , fmt='%.6f')
    header = "ref_q_iso   ref_i_iso"
    np.savetxt(os.path.join(sample_dir, f'ref_integration_{sample_name}.txt'), ref_integration_col,header = header , fmt='%.6f')
    header = "corr_q_iso   corr_i_iso"
    np.savetxt(os.path.join(sample_dir, f'corr_integration_{sample_name}.txt'), corr_integration_col,header = header , fmt='%.6f')


    #plot graph
     # Create the plot
    fig,ax = plt.subplots(figsize=(10, 6))
    ax.loglog(sample_q_iso, sample_i_iso, label='Sample I_iso', color='blue', linestyle='-')
    ax.loglog(ref_q_iso, ref_i_iso, label='Ref I_iso', color='red', linestyle='--')
    ax.loglog(sample_q_iso, corr_i_iso, label='Corr I_iso', color='green', linestyle=':')
    # Add labels and title
    ax.set_xlabel('q_iso')
    ax.set_ylabel('I_iso')
    ax.set_title(f' {sample_name}')
    ax.legend()
    # Save the figure
    plt.savefig(os.path.join(sample_dir, f'plot_iteration_{sample_name}.png'))
    plt.close(fig)  # Close the figure to free up memory

    



- open and plot

In [33]:
# Get a list of all the .npy files for samples, references, and corrections
sample_files = sorted([f for f in os.listdir(folder) if f.startswith('sample_integration_') and f.endswith('.npy')])
ref_files = sorted([f for f in os.listdir(folder) if f.startswith('ref_integration_') and f.endswith('.npy')])
corr_files = sorted([f for f in os.listdir(folder) if f.startswith('corr_integration_') and f.endswith('.npy')])
# Loop through all the files and plot the data
for index in range(len(sample_files)):
    # Load the data
    sample_data = np.load(os.path.join(folder, sample_files[index]))
    ref_data = np.load(os.path.join(folder, ref_files[index]))
    corr_data = np.load(os.path.join(folder, corr_files[index]))
    # Extract q_iso and I_iso for sample, ref, and corr
    sample_q_iso, sample_i_iso = sample_data[:, 0], sample_data[:, 1]
    ref_q_iso, ref_i_iso = ref_data[:, 0], ref_data[:, 1]
    corr_q_iso, corr_i_iso = corr_data[:, 0], corr_data[:, 1]
    # Create the plot
    fig,ax = plt.subplots(figsize=(10, 6))

    #plt.figure(figsize=(10, 6))
    ax.loglog(sample_q_iso, sample_i_iso, label='Sample I_iso', color='blue', linestyle='-')
    ax.loglog(ref_q_iso, ref_i_iso, label='Ref I_iso', color='red', linestyle='--')
    ax.loglog(corr_q_iso, corr_i_iso, label='Corr I_iso', color='green', linestyle=':')
    # Add labels and title
    ax.set_xlabel('q_iso')
    ax.set_ylabel('I_iso')
    ax.set_title(f'I_iso vs q_iso for iteration {index}')
    ax.legend()
    # Save the figure
    #plt.savefig(os.path.join(folder, f'plot_iteration_{index}.png'))
    #plt.savefig(os.path.join(sample_dir, f'plot_iteration_{index}.png'))
    plt.savefig(os.path.join(sample_dir, f'plot_iteration_{index}.png'))
    #(os.path.join(sample_dir, f'sample_integration_{index}.npy'), sample_integration_col)
    plt.close(fig)  # Close the figure to free up memory

In [34]:
print(os.path.join(sample_dir, f'plot_iteration_{index}.png'))

D:/swing/test\S_C_2_SF_5_P_10\plot_iteration_2.png


In [4]:
print(f'Total entries in DataFrame: {len(df)}')

Total entries in DataFrame: 6


In [5]:
# Separate references and samples
references = df[df['Type'] == 'ref']
samples = df[df['Type'] == 'sample']
# Merge on Channel and Position
merged = pd.merge(samples, references, on=['Channel', 'Position'], suffixes=('_sample', '_ref'))


In [6]:
merged.head(300)

,File_sample,Sample name_sample,Mean Transmission_sample,Type_sample,Channel,Position,Time_sample,Flow_sample,File_ref,Sample name_ref,Mean Transmission_ref,Type_ref,Time_ref,Flow_ref


In [7]:
# Select relevant columns
result = merged[['File_sample', 'Sample name', 'Mean Transmission_sample', 'File_ref', 'Sample name_ref', 'Mean Transmission_ref']]

KeyError: "['Sample name'] not in index"

In [ ]:
# Assuming df is your original DataFrame
# Filter samples
df_samples = df[df['Type'] == 'sample']
# Extract the pattern from the Sample name
df_samples['ref_sample_name'] = df_samples['Sample name'].str[2:]
# Filter references
df_references = df[df['Type'] == 'ref']
# Merge samples with references
df_samples = pd.merge(df_samples, df_references[['Sample name', 'File']], 
                      left_on='ref_sample_name', right_on='Sample name', 
                      suffixes=('_sample', '_ref'))
# Rename columns
df_samples = df_samples.rename(columns={'Sample name_sample': 'Sample name', 
                                        'Sample name_ref': 'corresponding ref sample name',
                                        'File': 'corresponding ref file'})
# Select the desired columns
df_samples = df_samples[['File', 'Sample name', 'Mean Transmission', 'Type', 'Channel', 'Position', 
                         'Time', 'Flow', 'corresponding ref sample name', 'corresponding ref file']]
# Display the resulting DataFrame
print(df_samples)

In [ ]:
# # Adding the 'Type' column using if-else statements
# df['Type'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     if row['Sample name'].startswith('R'):
#         df.at[index, 'Type'] = 'ref'
#     elif row['Sample name'].startswith('S'):
#         df.at[index, 'Type'] = 'sample'

# # Adding the 'Channel' column
# df['Channel'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         channel_value = row['Sample name'].split('_C_')[1][0]
#         if channel_value in ['1', '2']:
#             df.at[index, 'Channel'] = int(channel_value)
#     except (IndexError, ValueError):
#         pass  # Default value is already 'unknown'

# # Adding the 'Position' column
# df['Position'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         position_value = row['Sample name'].split('_P_')[1].split('_')[0]
#         df.at[index, 'Position'] = int(position_value)
#     except (IndexError, ValueError):
#         pass  # Default value is already 'unknown'

# # Adding the 'Time' column
# df['Time'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         time_value = row['File'].split('.h5')[0].split('_')[-2:]  # Get the last two parts (date and time)
#         full_time_string = '-'.join(time_value)  # Combine them with a dash
#         df.at[index, 'Time'] = pd.to_datetime(full_time_string, format='%Y-%m-%d-%H-%M-%S')  # Convert to datetime
#     except (IndexError, ValueError):
#         pass  # Default value is already 'unknown'

# # Adding the 'Flow' column
# df['Flow'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         if 'SF' in row['Sample name']:
#             df.at[index, 'Flow'] = 'SF'
#         elif 'LF' in row['Sample name']:
#             df.at[index, 'Flow'] = 'LF'
#     except Exception as e:
#         pass  # Default value is already 'unknown'

# df.head(200)

In [ ]:
# # Adding the 'Type' column using if-else statements
# df['Type'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     if row['Sample name'].startswith('R'):
#         df.at[index, 'Type'] = 'ref'
#     elif row['Sample name'].startswith('S'):
#         df.at[index, 'Type'] = 'sample'

# df['Channel'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         if row['Sample name'].split('_C_')[1][0] == '1':
#             df.at[index, 'Channel'] = 1
#         if row['Sample name'].split('_C_')[1][0] == '2':
#              df.at[index, 'Channel'] = 2
       
#     except IndexError:
#         # Handle cases where '_C_' or '_P_' might not be present
#         df.at[index, 'Channel'] = 'unknown'


# df['Position'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         # Split on '_P_' and take the second part, then split on underscore or whitespace
#         position_value = row['Sample name'].split('_P_')[1].split('_')[0]
#         df.at[index, 'Position'] = int(position_value)  # Convert to integer
#     except IndexError:
#         # Handle cases where '_P_' might not be present
#         df.at[index, 'Position'] = 'unknown'

# df['Time'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:
#         # Extract the part of the filename before the '.h5'
#         time_value = row['File'].split('.h5')[0].split('_')[-2:]  # Get the last two parts (date and time)
#         full_time_string = '-'.join(time_value)  # Combine them with a dash
#         df.at[index, 'Time'] = pd.to_datetime(full_time_string, format='%Y-%m-%d-%H-%M-%S')  # Convert to datetime
#     except IndexError:
#         df.at[index, 'Time'] = 'unknown'

# df['Flow'] = 'unknown'  # Set default value
# for index, row in df.iterrows():
#     try:   
#     # Check if 'SF' or 'LF' is in the 'Sample name'
#         if 'SF' in row['Sample name']:
#             df.at[index, 'Flow'] = 'SF'
#         elif 'LF' in row['Sample name']:
#             df.at[index, 'Flow'] = 'LF'
#         else:
#             df.at[index, 'Flow'] = 'unknown'  # This line is optional since default is already 'unknown'
#     except Exception as e:
#         df.at[index, 'Flow'] = 'unknown'  # Handle any unexpected errors
# df.head(200)



In [ ]:
# Count occurrences of 'unknown' in the 'Type' column
unknown_count_type = df['Type'].value_counts().get('unknown', 0)

# Count occurrences of 'unknown' in the 'Channel' column
unknown_count_channel = df['Channel'].value_counts().get('unknown', 0)

# Count occurrences of 'unknown' in the 'Position' column
unknown_count_position = df['Position'].value_counts().get('unknown', 0)

# Count occurrences of 'unknown' in the 'Time' column
unknown_count_time = df['Time'].value_counts().get('unknown', 0)

print(f"Unknown counts in each column:")
print(f"Type: {unknown_count_type}")
print(f"Channel: {unknown_count_channel}")
print(f"Position: {unknown_count_position}")
print(f"Time: {unknown_count_time}")

- Extract data from *.h5 file

In [4]:
# data_folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
# file = data_folder+'rodriguez_10448_2024-10-14_02-17-36.h5'

In [ ]:
# sample_name, eiger, eiger_mean, basler_image, position, params, time_stamps, transmission = extract(file,print_output=True)

In [4]:
# maskfile = 'mask.edf'
# integration_mean = integ(eiger_mean,params,maskfile)

- Plot and/or save plot

In [5]:
# plot_integration(integration_mean,eiger_mean,sample_name,data_folder=data_folder,plot=False,save_path=data_folder)

- Save data to files

In [ ]:
# save_exported_data(file,sample_name, eiger, basler_image, position, params, time_stamps, transmission, integration_mean ,maskfile)


- faire tourner en boucle
- faire code création fichier recap nom/fichier/transmission Pour correction d'intensité
- tester sur :
    - capillaire
    - Kin_21_Au500tes
    - Kin_11

- Run on all the files in the folder

In [ ]:
# data_folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
# # Get a list of all files in the directory
# files = [f for f in os.listdir(data_folder) if f.endswith('.h5')]
# maskfile = 'mask.edf'
# # Loop over the files
# for file in files:
#     try:
#         print(file)
#         # file_path = os.path.join(data_folder, file)
#         # sample_name, eiger, eiger_mean, basler_image, position, params, time_stamps, transmission = extract(file_path, print_output=False)
#         # integration_mean = integ(eiger_mean,params,maskfile)
#         # save_exported_data(file,sample_name, eiger, basler_image, position, params, time_stamps, transmission, integration_mean ,maskfile)
#     except Exception as e:
#         error_message = f"Error processing file {file}: {e}"
#         print(error_message)  # Print to console
#         logging.error(error_message)  # Log the error to the log file
    

In [11]:
# data_folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
# file = data_folder+'rodriguez_10448_2024-10-14_02-17-36.h5'
# maskfile = 'mask.edf'
# sample_name, eiger, eiger_mean, basler_image, position, params, time_stamps, transmission = extract(file, print_output=False)
# integration_mean = integ(eiger_mean,params,maskfile)
# save_exported_data(file,sample_name, eiger, basler_image, position, params, time_stamps, transmission, integration_mean ,maskfile)


In [14]:

# def batch_convert(data_folder,maskfile):
#     """
#     Processes all .h5 files in the specified data folder, performing data extraction,
#     integration, and saving of the results. Additionally, it copies all .png files from
#     the data folder and its subfolders to a 'plots' directory.
#     Parameters:
#     - data_folder (str): The path to the folder containing the .h5 files and subfolders.
#     - maskfile (str): The path to the mask file used for integration.
#     Returns:
#     None
#     Example:
#     >>> data_folder = 'C:/path/to/data_folder/'
#     >>> maskfile = 'C:/path/to/mask.edf'
#     >>> batch_convert(data_folder, maskfile)
#     This will process all .h5 files in the specified folder and copy all .png files to a 'plots' subfolder.
#     """
#     # Get a list of all files in the directory
#     files = [f for f in os.listdir(data_folder) if f.endswith('.h5')]
#     # Loop over the files
#     for index, file in enumerate(files):
#         try:
#             print(f'Processing file {index + 1}/{len(files)}: {file}')
#             clear_output(wait=True)
#             file_path = os.path.join(data_folder, file)  # Construct the full path for each file
#             # Extract data from the file
#             sample_name, eiger, eiger_mean, basler_image, position, params, time_stamps, transmission = extract(file_path, print_output=False)
#             # Perform integration
#             integration_mean = integ(eiger_mean, params, maskfile)
#             # Save the exported data
#             save_exported_data(file_path, sample_name, eiger, basler_image, position, params, time_stamps, transmission, integration_mean, maskfile)
#         except Exception as e:
#             print(f"Error processing file {file}: {e}")

#     #copy images of plots to have together
#     plot_folder = data_folder+'plots'
#     os.makedirs(plot_folder, exist_ok=True)
#         # Walk through the source folder and its subfolders
#     for root, dirs, files in os.walk(data_folder):
#         for file in files:
#             # Skip the plot folder
#             if root == plot_folder:
#                 continue
#             if file.endswith('.png'):
#                 # Construct full file path
#                 file_path = os.path.join(root, file)
#                 # Copy the file to the destination folder
#                 shutil.copy(file_path, plot_folder)


In [ ]:

# data_folder = 'C:/Users/cayez/Documents/DONNEES_SIMON/LPCNO/Synchrotron/SoleilSWING_octobre24/Notebook_traitement/dryed_cappilary/'
# maskfile = 'mask.edf'
# batch_convert(data_folder,maskfile)